In [ ]:
%pip install unstructured libmagic python-magic langchain tiktoken

In [ ]:
%pip install langchain-google-genai langsmith langserve langgraph python-dotenv

Import Libraries and Environment variables

In [ ]:
import langchain
import time
import pickle
import os
from dotenv import load_dotenv
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain.document_loaders import UnstructuredURLLoader
from langchain.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

load_dotenv()



In [ ]:
loader = UnstructuredURLLoader(urls=[
    "https://www.siemens.com/global/en/products/services/gbs/operations/digital-solutions.html",
    "https://www.siemens.com/global/en/products/services/gbs/opportunity-to-cash.html"
])
data = loader.load()

In [ ]:


text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20, length_function=len)

In [ ]:
texts = text_splitter.split_documents(data)

In [ ]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

vectorindex_openai = FAISS.from_documents(texts, embeddings)